In [36]:
!pip install pandas scikit-learn nltk

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [38]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
df = pd.read_excel("D:\DELL8\Documents\pharmacy_questions.xlsx")

In [40]:
df.head()

,Questions,Answers
0,What are your pharmacy hours?,Our pharmacy hours are 9 AM to 9 PM from Mond...
1,Do you offer delivery services?,"Yes, we offer delivery services. You can plac..."
2,Can I refill my prescription online?,"Yes, you can refill your prescription online ..."
3,What insurance plans do you accept?,We accept a wide range of insurance plans. Pl...
4,Do you offer flu shots?,"Yes, we offer flu shots. You can walk in or s..."


In [41]:
df_required_questions= df['Questions']
df_required_answers=df['Answers']

In [42]:
df_required_answers

0      Our pharmacy hours are 9 AM to 9 PM from Mond...
1      Yes, we offer delivery services. You can plac...
2      Yes, you can refill your prescription online ...
3      We accept a wide range of insurance plans. Pl...
4      Yes, we offer flu shots. You can walk in or s...
                            ...                        
95     Yes, you can get a prescription for acne trea...
96     Yes, we offer heart health products. Ask our ...
97     If your medication is recalled, we will notif...
98     Yes, you can get a prescription for anti-infl...
99     Yes, we offer wound care products. Ask our ph...
Name: Answers, Length: 100, dtype: object

In [43]:
# Filter empty strings directly
required_questions = []

for question in df_required_questions:
    if question.strip():  # Skip empty or whitespace-only entries
        data = question.split('.', 1)
        if len(data) > 1 and data[1].strip():  # Only keep meaningful content
            required_questions.append(data[1].strip())
        else:
            required_questions.append(question.strip())  

# Same logic for answers
required_answers = []

for answer in df_required_answers:
    if answer.strip():  # Skip empty or whitespace-only entries
        data = answer.split('.', 1)
        if len(data) > 1 and data[1].strip():
            required_answers.append(data[1].strip())
        else:
            required_answers.append(answer.strip())


In [44]:
required_questions


['What are your pharmacy hours?',
 'Do you offer delivery services?',
 'Can I refill my prescription online?',
 'What insurance plans do you accept?',
 'Do you offer flu shots?',
 'What is your return policy on medications?',
 'How can I transfer my prescription to your pharmacy?',
 'Do you have a loyalty program?',
 'Can I speak to a pharmacist?',
 'What COVID-19 precautions are you taking?',
 'How can I check if my medication is in stock?',
 'Do you offer a consultation service?',
 'Can I get my blood pressure checked at your pharmacy?',
 'Do you have a drive-thru service?',
 'What is the process for getting a new prescription filled?',
 'Can I get a copy of my prescription history?',
 'Do you offer discounts for senior citizens?',
 'How do I set up an online account for prescription refills?',
 'Do you provide medication synchronization services?',
 'What are your holiday hours?',
 'Can I order over-the-counter medications online?',
 'Do you offer travel vaccinations?',
 'How can I 

In [45]:
required_answers


['Our pharmacy hours are 9 AM to 9 PM from Monday to Saturday, and 10 AM to 6 PM on Sunday.',
 'You can place an order online or call us.',
 'Yes, you can refill your prescription online through our website.',
 'Please check our website or call us for more details.',
 'You can walk in or schedule an appointment.',
 'Our return policy on medications is that they can be returned within 14 days if unopened and in original condition.',
 'To transfer your prescription, please bring your prescription bottle or call us with the details.',
 'Sign up in-store or online to start earning rewards.',
 'Just call or visit our store.',
 'We are following all recommended COVID-19 precautions, including mask-wearing, social distancing, and regular sanitization.',
 'You can check if your medication is in stock by calling us or checking our website.',
 'You can schedule an appointment online or in-store.',
 'Yes, you can get your blood pressure checked at our pharmacy.',
 'Yes, we have a drive-thru servi

In [46]:
df.head()

,Questions,Answers
0,What are your pharmacy hours?,Our pharmacy hours are 9 AM to 9 PM from Mond...
1,Do you offer delivery services?,"Yes, we offer delivery services. You can plac..."
2,Can I refill my prescription online?,"Yes, you can refill your prescription online ..."
3,What insurance plans do you accept?,We accept a wide range of insurance plans. Pl...
4,Do you offer flu shots?,"Yes, we offer flu shots. You can walk in or s..."


In [47]:
df['Questions']=required_questions
df['Answers']=required_answers

In [48]:
df.head()

,Questions,Answers
0,What are your pharmacy hours?,Our pharmacy hours are 9 AM to 9 PM from Monda...
1,Do you offer delivery services?,You can place an order online or call us.
2,Can I refill my prescription online?,"Yes, you can refill your prescription online t..."
3,What insurance plans do you accept?,Please check our website or call us for more d...
4,Do you offer flu shots?,You can walk in or schedule an appointment.


In [49]:
df['Questions']

0                         What are your pharmacy hours?
1                       Do you offer delivery services?
2                  Can I refill my prescription online?
3                   What insurance plans do you accept?
4                               Do you offer flu shots?
                            ...                        
95         Can I get a prescription for acne treatment?
96                  Do you offer heart health products?
97       What should I do if my medication is recalled?
98    Can I get a prescription for anti-inflammatory...
99                    Do you offer wound care products?
Name: Questions, Length: 100, dtype: object

In [50]:
import spacy

# Load SpaCy's English model
nlp = spacy.load("en_core_web_sm")

def preprocess_text_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_punct and not token.is_stop]
    return ' '.join(tokens)


In [51]:

df['processed_question'] = df['Questions'].apply(preprocess_text_spacy)



In [52]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['processed_question'])

In [61]:
def get_answer(user_question):
    processed_user_question = preprocess_text_spacy(user_question)
    user_tfidf = vectorizer.transform([processed_user_question])
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    
    print(type(cosine_similarities))
    sim = True
    print()
    
    if max(cosine_similarities) >= 0.75:
        most_similar_index = cosine_similarities.argmax()
        print(most_similar_index)
        data = {
            'answer': df.iloc[most_similar_index]['Answers'],
            'question': df.iloc[most_similar_index]['Questions'],
            'sim': sim
        }
    else:
        sim = False
        most_similar_index = cosine_similarities.argmax()
        print(most_similar_index)
        data = {
            'question': df.iloc[most_similar_index]['Questions'],
            'Text': 'You can try another question like this',
            'sim': sim
        }
    return data

In [62]:
while True:
    user_question = input("\nAsk me a question (or type 'exit' to quit): ")
    if user_question.lower() == 'exit':
        print("Thank you! Have a great day!")
        break

    data = get_answer(user_question)

    print("\n" + "=" * 60)
    if data['sim']:
        print(f"Matched Question: {data['question']}")
        print(f"Answer: {data['answer']}")
    else:
        print("No exact match found.")
        print("Try asking like this:")
        print(f"Example: {data['question']}")
    print("=" * 60 + "\n")



Ask me a question (or type 'exit' to quit):  insurace plan


<class 'numpy.ndarray'>

0

No exact match found.
Try asking like this:
Example: What are your pharmacy hours?




Ask me a question (or type 'exit' to quit):  insurance plan


<class 'numpy.ndarray'>

3

No exact match found.
Try asking like this:
Example: What insurance plans do you accept?




Ask me a question (or type 'exit' to quit):   insurance plans


<class 'numpy.ndarray'>

3

Matched Question: What insurance plans do you accept?
Answer: Please check our website or call us for more details.




Ask me a question (or type 'exit' to quit):  exit


Thank you! Have a great day!
